In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf
import PIL
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import os
import pandas as pd


TEST_DIR = '/content/test/test'
TRAIN_DIR = '/content/train/train'

train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size=32,
    class_mode='categorical',
    target_size=(224,224))

conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)

optimizer = keras.optimizers.Adam()
from keras.optimizers import SGD
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(generator=train_generator,
                              samples_per_epoch = 3138 // 32,  # added in Kaggle
                              epochs = 20,
                              validation_data=0,
                              validation_steps=10  # added in Kaggle
                             )



testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")